In [13]:
# Import libraries
import numpy as np
import pandas as pd
import pymysql
from sklearn import (metrics, linear_model)
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
# Connect to PIC
con = pymysql.connect(host='localhost',
                             user='root',
                             password='13163232289zx',
                             db='pmimic',
                             cursorclass=pymysql.cursors.DictCursor)
cur=con.cursor()

In [3]:
#patient characteristics
query = \
"""
WITH patientcharac as (
SELECT a.subject_id, a.hadm_id, i.icustay_id, 
    a.admittime as hosp_admittime, a.dischtime as hosp_dischtime, 
    a.hospital_expire_flag,i.first_careunit, 
    p.dob,i.intime as icu_intime, i.outtime as icu_outtime, 
    round(timestampdiff(SECOND,i.intime,i.outtime)/60/60/24,4) as icu_los, 
    p.gender, 
    round(timestampdiff(SECOND,p.dob,a.admittime)/60/60/24/365,4) as age_hosp_in
FROM admissions a
INNER JOIN icustays i
ON a.hadm_id = i.hadm_id
INNER JOIN patients p
ON a.subject_id = p.subject_id
)
SELECT * from patientcharac
where hospital_expire_flag=0;
"""

patient_characteristics = pd.read_sql_query(query,con)
print(patient_characteristics.head())

   subject_id  hadm_id  icustay_id      hosp_admittime      hosp_dischtime  \
0          26   100000      200532 2098-11-09 18:30:55 2098-11-14 09:18:00   
1          29   100002      200534 2062-11-29 20:52:52 2062-12-07 14:27:00   
2          32   100005      200537 2105-09-17 09:59:36 2105-10-01 10:17:00   
3          23   100006      200538 2108-09-29 18:35:01 2108-10-10 11:24:00   
4          35   100007      200539 2091-11-04 12:40:40 2091-11-15 17:02:00   

   hospital_expire_flag first_careunit                 dob  \
0                     0    General ICU 2098-06-22 00:00:00   
1                     0    General ICU 2060-01-22 00:00:00   
2                     0    General ICU 2105-07-22 00:00:00   
3                     0    General ICU 2108-04-15 05:21:00   
4                     0    General ICU 2080-01-12 00:00:00   

           icu_intime         icu_outtime  icu_los gender  age_hosp_in  
0 2098-11-09 18:30:55 2098-11-14 09:18:00   4.6160      F       0.3857  
1 2062-11-29

In [9]:
#delete patients who had multiple ICU stay during a hospital stay
query1 = \
"""
WITH multiplestay AS (
SELECT hadm_id, count(hadm_id) as obs
FROM icustays
GROUP BY hadm_id)
SELECT *
FROM multiplestay
where obs>1;
"""
multiplestay = pd.read_sql_query(query1,con)
patient_characteristics=patient_characteristics[~patient_characteristics.hadm_id.isin(multiplestay.hadm_id)]

In [10]:
#vital_signs
query1 = \
"""
SELECT *
FROM chartevents;
"""

vital_sign = pd.read_sql_query(query1,con)
vital_sign=vital_sign[vital_sign.ITEMID.isin(['1001','1002','1003','1015','1016'])]
patient_characteristics=patient_characteristics[patient_characteristics.hadm_id.isin(vital_sign.HADM_ID)]
def dataprocess(x):
    data=vital_sign[vital_sign.HADM_ID==x]
    temperature_mean=data[data.ITEMID=='1001'].VALUENUM.mean()
    temperature_std=data[data.ITEMID=='1001'].VALUENUM.std()
    pulse_mean=data[data.ITEMID=='1002'].VALUENUM.mean()
    pulse_std=data[data.ITEMID=='1002'].VALUENUM.std()
    heartrate_mean=data[data.ITEMID=='1003'].VALUENUM.mean()
    heartrate_std=data[data.ITEMID=='1003'].VALUENUM.std()
    SBP_mean=data[data.ITEMID=='1016'].VALUENUM.mean()
    SBP_std=data[data.ITEMID=='1016'].VALUENUM.std()
    DBP_mean=data[data.ITEMID=='1015'].VALUENUM.mean()
    DBP_std=data[data.ITEMID=='1015'].VALUENUM.std()
    return temperature_mean,temperature_std,pulse_mean,pulse_std,heartrate_mean,heartrate_std,SBP_mean,SBP_std,DBP_mean,DBP_std
patient_characteristics['vital_sign']=patient_characteristics['hadm_id'].apply(dataprocess)

In [12]:
patient_characteristics['temperature_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[0]
patient_characteristics['temperature_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[1]
patient_characteristics['pulse_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[2]
patient_characteristics['pulse_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[3]
patient_characteristics['heartrate_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[4]
patient_characteristics['heartrate_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[5]
patient_characteristics['SBP_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[6]
patient_characteristics['SBP_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[7]
patient_characteristics['DBP_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[8]
patient_characteristics['DBP_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[9]
patient_characteristics['temperature_mean']=patient_characteristics['temperature_mean'].apply(lambda x:x[1:])
patient_characteristics['DBP_std']=patient_characteristics['DBP_std'].apply(lambda x:x[:len(x)-1])
def processstring(x):
    if x==' nan':
        x=np.nan
    else:
        x=round(float(x),2)
    return x
for c in ['temperature_mean','temperature_std','pulse_mean','pulse_std','heartrate_mean'
          ,'heartrate_std','SBP_mean','SBP_std','DBP_mean','DBP_std']:
    patient_characteristics[c]=patient_characteristics[c].apply(processstring)
def processgender(x):
    if x=='F':
        x='1'
    else:
        x='0'
    x=int(x)
    return x
patient_characteristics['gender']=patient_characteristics['gender'].apply(processgender)

In [15]:
#xgboost prediction model
X=patient_characteristics[['gender','age_hosp_in','temperature_mean','temperature_std','pulse_mean','pulse_std','heartrate_mean'
          ,'heartrate_std','SBP_mean','SBP_std','DBP_mean','DBP_std']]
y=patient_characteristics['icu_los']
a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf_xgb=xgb.XGBRegressor(booster='gbtree',objective="reg:linear")
clf_xgb.fit(a_train, b_train)
prediction_xgb=clf_xgb.predict(a_test)
print('\nMean_absolute_error: {}'\
    .format( metrics.mean_absolute_error(b_test, prediction_xgb)))
print('\nExplained_variance_score: {}'\
    .format( metrics.explained_variance_score(b_test, prediction_xgb)))


Mean_absolute_error: 6.376199898192746

Explained_variance_score: 0.39886204568062733
